In [ ]:
#Crawling National Fire Stastistics from NFDS(National Fire Data System) by manipulating Javascript using Selenium and PhantomJS

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import datetime
import pandas as pd

In [2]:
#This crawler tested Chrome, PhantomJS(headless browser), and Firefox to implement crawler
path_chrome = r"C:\Users\Administrator\Anaconda3\chromedriver_win32\chromedriver.exe"
path_js = r"C:\Users\Administrator\Anaconda3\phantomjs-2.1.1-windows\bin\phantomjs.exe"
path_gecko = r"C:/Users/Administrator/Anaconda3/geckodriver-v0.19.1-win64/geckodriver.exe"

In [205]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')
chrome = webdriver.Chrome(executable_path=path_chrome, chrome_options=options)

In [3]:
firefox = webdriver.Firefox(executable_path=path_gecko)

In [6]:
js = webdriver.PhantomJS(path_js)

In [4]:
#URL of fire data categorized by Province
url_area= "http://nfds.go.kr/fr_base_0001.jsf"
table_selector_area = '#iTable > table > tbody > tr'
xpath_area ='//*[@id="content-area"]/div[2]/div[2]/form/table/tbody/tr/td[2]/select[1]'
columns_area=['구분','계','실화_건수','자연적요인_건수','방화_건수','미상_건수','실화_비율',
'자연적요인_비율','방화_비율','미상_비율','인명피해_계','인명피해_사망','인명피해_부상',
'재산피해_계','재산피해_부동산','재산피해_동산','소실면석_동수','소실면적_면적','도','날짜']

In [5]:
#URL of fire data categorized by Fire Station
url_station ="http://nfds.go.kr/fr_base_0101.jsf"
table_selector_station = '#iTable > table > tbody > tr'
xpath_station_1 = '//*[@id="slFireCenter1"]'
columns_station=['구분','계','실화_건','자연적요인_건','방화_건','미상_건','실화_비율',
                '자연적요인_비율','방화_비율','미상_비율','인명피해_계','인명피해_사망',
                '인명피해_부상','재산피해_계','재산피해_부동산','재산피해_동산',
                '소실면적_동수','소실면적_면적','본부','날짜']

In [6]:
#URL of fire data categorized by Fire Prevention System
url_facility = 'http://nfds.go.kr/fr_pos_0001.jsf'
#When parsing the tables Table, I skipped the first row because it contained useless content
table_selector_facility_1 = '#iTable > table > tbody > tr > td:nth-of-type(1) > table > tbody > tr'
table_selector_facility_2 = '#tbHeadDiv > table > tbody > tr'
type_xpath_facility='//*[@id="content-area"]/div[2]/div[2]/form/table/tbody/tr[1]/td[2]/select'
xpath_facility_1='//*[@id="slFireCenter1"]'
xpath_facility_2='//*[@id="slFireCenter2"]'
columns_facility =['구분_대','구분_소','합계','소화기구_계','소화기구_사용','소화기구_미사용',
                 '소화기구_미상','옥내소화전_계','옥내소화전_사용','옥내소화전_미사용/효과미비','옥내소화전_미상',
                 '스프링클러등_계','스프링쿨러등_효과적작용','스프링쿨러등_소규모화재로미작동','스프링쿨러등_미작동/효과없음',
                 '비상경보설비_계','비상경보설비_경보','비상경보설비_미경보','비상경보설비_미상',
                 '자동화재탐지설비_계','자동화재탐지설비_작동','자동화재탐지설비_소규모화재로미작동','자동화재탐지설비_미상',
                 '방화문_계','방화문_정상','방화문_비정상','방화구획_계','방화구획_설치','방화구획_미설치','데이터종류','본부','소방서','날짜']

In [7]:
#URL of fire data categorized by PUBLICLY USED ESTABLISHMENTS
url_multi = "http://nfds.go.kr/fr_pos_0201.jsf"
table_selector_multi = '#iTable > table > tbody > tr'
type_xpath_multi='//*[@id="content-area"]/div[2]/div[2]/form/table/tbody/tr[1]/td[2]/select'
xpath_multi_1='//*[@id="slFireCenter1"]'
xpath_multi_2='//*[@id="slFireCenter2"]'
columns_multi=['구분_대','구분_소','계','전기적','기계적','화학적','가스누출','교통사고',
             '부주의','기타','자연적요인','방화','방화의심','미상','데이터종류','본부','소방서','날짜']

In [8]:
#URL of fire data categorized by building structure
url_bldg="http://nfds.go.kr/fr_pos_0401.jsf"
table_selector_bldg = '#iTable > table > tbody > tr'
type_xpath_bldg = '//*[@id="content-area"]/div[2]/div[2]/form/table/tbody/tr[1]/td[2]/select'
xpath_bldg_1 = '//*[@id="slFireCenter1"]'
xpath_bldg_2 = '//*[@id="slFireCenter2"]'
columns_bldg = ['구분','계','전기적','기계적','화학적요인','가스누출','교통사고','부주의',
              '기타','자연적요인','방화','방화의심','미상','구분','본부','소방서','날짜']

In [9]:
#URL of fire data categorized by cause of fire
url_rsn_lc = 'http://nfds.go.kr/fr_fact_0001.jsf'
table_selector_rsn_lc = '#iTable > table > tbody > tr'
type_xpath_rsn_lc = '//*[@id="content-area"]/div[2]/div[2]/form/table/tbody/tr[1]/td[2]/select'
xpath_rsn_lc_1 = '//*[@id="slFireCenter1"]'
xpath_rsn_lc_2 = '//*[@id="slFireCenter2"]'
columns_rsn_lc=['구분_대','구분_소','계','전기적','기계적','화학적요인','가스누출','교통사고',
              '부주의','기타','자연적요인','방화','방화의심','미상','구분','본부','소방서','날짜']

In [10]:
#URL of fire data categorized by fire spread cause
url_aggv_rsn = "http://nfds.go.kr/fr_pos_0101.jsf"
table_selector_aggv_rsn = '#iTable > table > tbody > tr'
type_xpath_aggv_rsn = '//*[@id="content-area"]/div[2]/div[2]/form/table/tbody/tr[1]/td[2]/select'
xpath_aggv_rsn_1 = '//*[@id="slFireCenter1"]'
xpath_aggv_rsn_2 = '//*[@id="slFireCenter2"]'
columns_aggv_rsn=['구분_대','구분_소','계','전기적','화학적요인','가스누출','교통사고','부주의',
                '기타','자연적요인','방화','방화의심','미상','구분','본부','소방서','날짜']

In [11]:
center_name=['서울소방재난본부','부산소방안전본부','대구소방안전본부','인천소방안전본부','광주소방본부',
            '대전소방본부','울산소방본부','세종소방본부','경기재난안전본부','경기북부소방재난본부','강원소방본부','충북소방본부',
            '충청남도소방본부','전북소방본부','전남소방본부','경북소방본부','경남소방본부','창원소방본부','제주소방본부']


In [12]:
station_name=[['종로소방서','중부소방서','광진소방서','용산소방서','동대문소방서','영등포소방서','성북소방서','은평소방서','강남소방서','서초소방서','강서소방서','강동소방서','마포소방서','도봉소방서','구로소방서','노원소방서','관악소방서','송파소방서','양천소방서','중랑소방서','동작소방서','서대문소방서','강북소방서','성동소방서'],
             ['중부소방서','부산진소방서','동래소방서','북부소방서','사하소방서','해운대소방서','금정소방서','남부소방서','강서소방서','항만소방서','기장소방서'],
             ['중부소방서','동부소방서','서부소방서','북부소방서','수성소방서','달서소방서','달성소방서','강서소방서'],
             ['중부소방서','남동소방서','부평소방서','서부소방서','공단소방서','계양소방서','남부소방서','강화소방서','공항소방서','송도소방서'],
             ['광산소방서','동부소방서','서부소방서','북부소방서','남부소방서'],
             ['중부소방서','서부소방서','동부소방서','북부소방서','남부소방서'],
             ['중부소방서','남부소방서','동부소방서','온산소방서'],
             ['조치원소방서','세종소방서'],
             ['수원소방서','성남소방서','분당소방서','부천소방서','안양소방서','안산소방서','용인소방서','평택소방서','송탄소방서','광명소방서','시흥소방서','군포소방서','화성소방서','이천소방서','김포소방서','광주소방서','안성소방서','하남소방서','의왕소방서','오산소방서','여주소방서','양평소방서','과천소방서','고양소방서','일산소방서','의정부소방서','남양주소방서','파주소방서','구리소방서','포천소방서','양주소방서','동두천소방서','가평소방서','연천소방서'],
             ['고양소방서','일산소방서','의정부소방서','남양주소방서','파주소방서','구리소방서','포천소방서','양주소방서','동두천소방서','가평소방서','연천소방서'],
              ['춘천소방서','원주소방서','강릉소방서','동해소방서','태백소방서','속초소방서','삼척소방서','홍천소방서','횡성소방서','영월소방서','평창소방서','정선소방서','철원소방서','인제소방서','고성소방서','양양소방서'],
              ['청주동부소방서','서부소방서','충주소방서','제천소방서','영동소방서','증평소방서','음성소방서','진천소방서','옥천소방서','보은소방서','괴산소방서','단양소방서'],
              ['천안동남소방서','천안서북소방서','공주소방서','보령소방서','아산소방서','서산소방서','논산소방서','당진소방서','금산소방서','부여소방서','서천소방서','홍성소방서','예산소방서','태안소방서','계룡소방서','청양소방서'],
              ['전주덕진소방서','전주완산소방서','군산소방서','익산소방서','정읍소방서','남원소방서','김제소방서','고창소방서','부안소방서','무진장소방서'],
              ['목포소방서','여수소방서','순천소방서','나주소방서','광양소방서','담양소방서','보성소방서','해남소방서','영암소방서','영광소방서','화순소방서','강진소방서','무안소방서','고흥소방서'],
              ['포항북부소방서','포항남부소방서','경주소방서','김천소방서','안동소방서','구미소방서','영주소방서','영천소방서','상주소방서','문경소방서','경산소방서','의성소방서','영덕소방서','청도소방서','고령소방서','성주소방서','칠곡소방서','울진소방서'],
              ['진주소방서','통영소방서','사천소방서','밀양소방서','거제소방서','양산소방서','의령소방서','함안소방서','창녕소방서','고성소방서','남해소방서','하동소방서','산청소방서','함양소방서','거창소방서','합천소방서','김해동부소방서','김해서부소방서'],
              ['창원소방본부','창원소방서','마산소방서'],
              ['제주소방서','서귀포소방서','서부소방서','동부소방서']
             ]

In [20]:
station_value=[['1103000,1104000','1105000','1106000',"1107000","1108000",'1109000','1112000','1114000','1115000','1117000','1118000','1113000','1110000','1119000','1111000','1116000','1120000','1121000','1122000','1102000','1123000','1125000','1127000'],
               ['2601000','2602000','2603000','2604000','2605000','2606000','2607000','2608000','2609000','2610000','2611000'],
               ['2701000','2702000','2703000','2704000','2707000','2705000','2706000','2708000'],
               ['2801000','2802000','2803000','2804000','2805000','2806000','2807000','2808000','2810000','2811000'],
               ['2901000','2902000','2903000','2904000','2905000'],
               ['3001000','3002000','3003000','3004000','3005000'],
               ['3101000','3102000','3103000','3104000'],
               ['3602000','3603000'],
               ['4137000','4104000','4105000','4107000','4108000','4109000','4110000','4112000','4113000','4114000','4115000','4116000','4133000','4118000','4120000','4126000','4122000','4123000','4132000','4124000','4127000','4128000','4101000','4106000','4129000','4111000','4130000','4117000','4119000','4121000','4154000','4125000','4150000','4152000'],
               ['4106000','4129000','4111000','4130000','4117000','4119000','4121000','4154000','4125000','4150000','4152000'],
               ['4201000','4202000','4203000','4204000','4205000','4206000','4207000','4208000','4213000','4209000','4214000','4211000','4210000','4215000','4216000','4217000'],
               ['4301000','4302000','4303000','4304000','4305000','4306000','4307000','4308000','4309000','4310000','4311000','4312000'],
               ['4402000','4416000','4403000','4404000','4405000','4406000','4407000','4409000','4412000','4410000','4413000','4401000','4408000','4417000','4421000','4420000'],
               ['4501000','4502000','4503000','4504000','4505000','4506000','4507000','4508000','4509000','4510000'],
               ['4601000','4602000','4603000','4604000','4605000','4606000','4607000','4608000','4609000','4610000','4611000','4612000','4613000','4615000'],
               ['4715000','4716000','4703000','4704000','4705000','4706000','4707000','4708000','4709000','4710000','4711000','4714000','4718000','4719000','4717000','4713000','4701000','4712000'],
               ['4804000','4806000','4807000','4809000','4810000','4811000','4821000','4814000','4813000','4819000','4817000','4818000','4822000','4820000','4812000','4815000','4826000','4827000'],
               ['4823000','4824000','4825000'],
               ['5001000','5002000','5003000','5004000']
               ]

In [16]:
station_value=["1103000","2601000","2701000","2801000","2901000","3001000","3101000",
               "3602000","4137000","4106000","4201000","4301000","4402000","4501000",
               "4601000","4715000","4804000","4823000","5001000"]

In [13]:
class nfds_crawling :
    
    def __init__(self, driver, url, from_,to_) :
        self.master_area=[]
        self.driver=driver
        self.url = url
        self.from_ = from_
        self.to_ = to_
        
    def date_creation(self) :
        start = datetime.datetime.strptime(self.from_, '%Y-%m-%d').date()
        end = datetime.datetime.strptime(self.to_, '%Y-%m-%d').date()
        day_range = (end-start).days + 1
        dates=[]
        for i in range(day_range) :
            dates.append(start+datetime.timedelta(days=i))
            dates[i] = dates[i].strftime('%Y-%m-%d')
        self.dates = dates
        
    def submission(self) :
        submit = self.driver.find_element_by_class_name(name='submit')
        submit.submit()
        
    def multi_select(self, xpath, index) :
        elem = self.driver.find_element_by_xpath(xpath)
        select = Select(elem)                
        options = select.options[index].text
        select.select_by_index(index)        
        return options
    
    def option_len(self, xpath) :
        elem = self.driver.find_element_by_xpath(xpath)
        select = Select(elem)
        return len(select.options)
    
    def date_selection(self, date_obj) :
        elem_start = self.driver.find_element_by_xpath('//*[@id="txtSdate"]')
        elem_end = self.driver.find_element_by_xpath('//*[@id="txtEdate"]')
        elem_start.clear()
        elem_end.clear()
        elem_start.send_keys(date_obj)
        elem_end.send_keys(date_obj)
        
    def table_waiting(self, selector) :
        try :
            element_present = EC.presence_of_all_elements_located((By.CSS_SELECTOR, selector))
            WebDriverWait(self.driver, 5).until(element_present)
        except TimeoutException :
            print('Internet Connection maybe lost at table')
                
    def option_waiting(self, xpath, text_) :
        try :
            element_present = EC.text_to_be_present_in_element_value((By.XPATH, xpath),text_)
            WebDriverWait(self.driver, 5).until(element_present)
        except TimeoutException :
            print('Internet Connection maybe lost at option')

    def option_staleness(self,elem) : 
        try :            
            element_present = EC.staleness_of(element=elem)
            WebDriverWait(self.driver, 5).until(element_present)
        except TimeoutException :
            print('Option has not property populated')
            
             
    #Parsing Table Function
    def parsing_table(self,date_obj,table_selector,*args) :
        html = self.driver.page_source
        soup = BeautifulSoup(html,'lxml')
        Soup = soup.select(table_selector)        
        for i in range(len(Soup)) :
            temp = Soup[i].get_text(separator='|', strip=True)
            #str인 temp를 list 작업화
            temp = temp.split('|')
            temp.extend(list(args))
            temp.append(date_obj)
            self.master_area.append(temp)
            
    # Parsing Table divided into 2 section
    def parsing_two_table(self,date_obj,table_selector_1,table_selector_2,*args) :
        html = self.driver.page_source
        soup = BeautifulSoup(html,'lxml')
        Soup_main = soup.select(table_selector_1)
        Soup_body = soup.select(table_selector_2)
    
        for i in range(1,len(Soup_main)) :
            temp_main = Soup_main[i].get_text(separator='|',strip=True)
            temp_main = temp_main.split('|')
            temp_body = Soup_body[i].get_text(separator='|',strip=True)
            temp_body = temp_body.split('|')            
            temp_main.extend(temp_body)
            temp_main.extend(list(args))
            temp_main.append(date_obj)
            self.master_area.append(temp_main)
        
            
    # 기본현황 - 시/도 (옵션 1개)
    def area_table(self,area_xpath,table_selector) :
        #GET URL
        self.driver.get(self.url)
        #SELECTING DATE
        for i in range(len(self.dates)) :
            self.date_selection(self.dates[i])
            # SELECTING FIRST OPTION
            for j in range(1, getattr(self,'option_len')(area_xpath)):
                option = getattr(self,'multi_select')(area_xpath,j)
                self.submission()
                getattr(self,'table_waiting')(table_selector)                
                getattr(self,'parsing_table')(self.dates[i],table_selector,option)
                
        return self.master_area
    
    # 기본현황 - 소방관서 (옵션 1개)
    def fire_station(self, area_xpath_1,table_selector) :
        #소방관서는 화재내역이 없을 경우 Table CSS Selector에 해당되는 테이블이
        #아예 나타나지 않기에 해당 경우를 인터넷 연결이 끊긴 것과 별도로
        #고려해야 함
        #GET URL
        self.driver.get(self.url)
        # SELECTING DATE
        for i in range(len(self.dates)):
            self.date_selection(self.dates[i])
            #SELECTING FIRST OPTION
            for j in range(1,getattr(self,'option_len')(area_xpath_1)) :
                option_a = getattr(self,'multi_select')(area_xpath_1,j)
                self.submission()
                getattr(self,'table_waiting')(table_selector)
                getattr(self,'parsing_table')(self.dates[i],table_selector,option_a)                
        
        return self.master_area
    
    # 장소별-소방방화시설 (옵션 3개)
    # 유일하게 두개의 table로 이루어졌기에 parsing_two_table 함수를 이용
    def fire_facility(self,type_xpath,xpath_1,xpath_2,table_selector_1,table_selector_2) :
        #GET URL
        self.driver.get(self.url)
        #SELECTING DATE
        for i in range(len(self.dates)):
            self.date_selection(self.dates[i])
            #SELECTING TYPE(발생건수,비율,피해)
            #2번째(j=1) 는 발생비율이므로 skip
            for j in [0,2,3,4,5,6,7] :
                option_a = getattr(self, 'multi_select')(type_xpath,j)
                # 소방본부 선택
                for k in range(1,getattr(self, 'option_len')(xpath_1)):
                    elem = self.driver.find_element_by_xpath(xpath_2+'/option[1]')
                    option_b=getattr(self,'multi_select')(xpath_1,k)
                    getattr(self,'option_waiting')(xpath_2+'/option[2]',station_value[k-1])
                    #소방서 선택
                    for m in range(1,getattr(self,'option_len')(xpath_2)) :                                                
                        option_c=getattr(self,'multi_select')(xpath_2,m)                        
                        self.submission()
                        getattr(self,'table_waiting')(table_selector_1)
                        getattr(self,'table_waiting')(table_selector_2)
                        getattr(self,'parsing_two_table')(self.dates[i],table_selector_1,table_selector_2,
                                                          option_a,option_b,option_c)

        return self.master_area
    
    # 장소별-다중이용업시설 (옵션 3개)
    def fire_multi(self,type_xpath,xpath_1,xpath_2,table_selector) :
        #GET URL
        self.driver.get(self.url)
        #SELECTING DATE
        for i in range(len(self.dates)) :
            self.date_selection(self.dates[i])
        #SELECTING DATA TYPE(2번쨰는 발생비율로 SKIP)
            for j in [0,2,3,4,5,6,7] :
                option_a = getattr(self,'multi_select')(type_xpath,j)
                for k in range(1,getattr(self,'option_len')(xpath_1)) :                    
                    option_b = getattr(self,'multi_select')(xpath_1,k)                    
                    getattr(self,'option_waiting')(xpath_2+'/option[2]',station_value[k-1])                    
                    for m in range(1,getattr(self,'option_len')(xpath_2)) :
                        option_c = getattr(self,'multi_select')(xpath_2,m)
                        self.submission()
                        getattr(self,'table_waiting')(table_selector)
                        getattr(self,'parsing_table')(self.dates[i],table_selector,
                                                     option_a,option_b,option_c)
        return self.master_area
    
    # 장소별-건물구조별
    def fire_bldg(self,type_xpath,xpath_1,xpath_2,table_selector) :
        #GET URL
        self.driver.get(self.url)
        #SELECTING DATE
        for i in range(len(self.dates)) :
            self.date_selection(self.dates[i])
        #SELECTING DATA TYPE(2번쨰는 발생비율로 SKIP)
            for j in [0,2,3,4,5,6,7] :
                option_a = getattr(self,'multi_select')(type_xpath,j)
                for k in range(1,getattr(self,'option_len')(xpath_1)) :
                    option_b = getattr(self,'multi_select')(xpath_1,k)
                    getattr(self,'option_waiting')(xpath_2+'/option[2]',station_value[k-1])
                    for m in range(1,getattr(self,'option_len')(xpath_2)) :
                        option_c = getattr(self,'multi_select')(xpath_2,m)
                        self.submission()
                        getattr(self,'table_waiting')(table_selector)
                        getattr(self,'parsing_table')(self.dates[i],table_selector,
                                                     option_a,option_b,option_c)
        return self.master_area
    
    #발화요인별-발화요인/장소별 (옵션 3개)
    def fire_rsn_lc(self,type_xpath,xpath_1,xpath_2,table_selector) :
        #GET URL
        self.driver.get(self.url)
        #SELECTING DATE
        for i in range(len(self.dates)) :
            self.date_selection(self.dates[i])
        #SELECTING DATA TYPE(2번쨰는 발생비율로 SKIP)
            for j in [0,2,3,4,5,6,7] :
                option_a = getattr(self,'multi_select')(type_xpath,j)
                for k in range(1,getattr(self,'option_len')(xpath_1)) :
                    option_b = getattr(self,'multi_select')(xpath_1,k)
                    getattr(self,'option_waiting')(xpath_2+'/option[2]',station_value[k-1])
                    for m in range(1,getattr(self,'option_len')(xpath_2)) :
                        option_c = getattr(self,'multi_select')(xpath_2,m)
                        self.submission()
                        getattr(self,'table_waiting')(table_selector)
                        getattr(self,'parsing_table')(self.dates[i],table_selector,
                                                     option_a,option_b,option_c)
        return self.master_area
    
    #발화요인별-연소확대사유현황 (옵션 3개)
    def fire_aggv_rsn(self,type_xpath,xpath_1,xpath_2,table_selector) :
        #GET URL
        self.driver.get(self.url)
        #SELECTING DATE
        for i in range(len(self.dates)) :
            self.date_selection(self.dates[i])
        #SELECTING DATA TYPE(2번쨰는 발생비율로 SKIP)
            for j in [0,2,3,4,5,6,7] :
                option_a = getattr(self,'multi_select')(type_xpath,j)
                for k in range(1,getattr(self,'option_len')(xpath_1)) :
                    option_b = getattr(self,'multi_select')(xpath_1,k)
                    getattr(self,'option_waiting')(xpath_2+'/option[2]',station_value[k-1])
                    for m in range(1,getattr(self,'option_len')(xpath_2)) :
                        option_c = getattr(self,'multi_select')(xpath_2,m)
                        self.submission()
                        getattr(self,'table_waiting')(table_selector)
                        getattr(self,'parsing_table')(self.dates[i],table_selector,
                                                     option_a,option_b,option_c)
        return self.master_area

In [ ]:
#연소확대사유 테스트
firefox.get(url_aggv_rsn)
test=nfds_crawling(firefox,url_aggv_rsn,'2017-09-01','2017-09-30')
test.date_creation()
test_body=test.fire_aggv_rsn(type_xpath_aggv_rsn,xpath_aggv_rsn_1,xpath_aggv_rsn_2,table_selector_aggv_rsn)
test_df = pd.DataFrame(test_body,columns=columns_aggv_rsn)
test_df.to_csv(index=False,encoding='utf-8-sig',path_or_buf=r'C:\Users\Administrator\Desktop\새 폴더\fire_aggv_rsn_20170901_20170930.csv')

In [ ]:
#발화요인 장소별 테스트
chrome.get(url_rsn_lc)
test=nfds_crawling(chrome,url_rsn_lc,'2017-09-01','2017-09-30')
test.date_creation()
test_body=test.fire_rsn_lc(type_xpath_rsn_lc,xpath_rsn_lc_1,xpath_rsn_lc_2,table_selector_rsn_lc)
test_df = pd.DataFrame(test_body,columns=columns_rsn_lc)
test_df.to_csv(index=False,encoding='utf-8-sig',path_or_buf=r'C:\Users\Administrator\Desktop\새 폴더\fire_rsn_lc_20170901_20170930.csv')

In [ ]:
#소방방화시설 테스트
firefox.get(url_facility)
test=nfds_crawling(firefox,url_facility,'2017-09-01','2017-09-30')
test.date_creation()
test_body=test.fire_facility(type_xpath_facility,xpath_facility_1,xpath_facility_2,table_selector_facility_1,table_selector_facility_2)
test_df = pd.DataFrame(test_body,columns=columns_facility)
test_df.to_csv(index=False,encoding='utf-8-sig',path_or_buf=r'C:\Users\Administrator\Desktop\새 폴더\fire_facility_20170901_20170930.csv')

In [ ]:
#다중이용시설 테스트
test=nfds_crawling(chrome,url_multi,'2017-09-01','2017-09-30')
test.date_creation()
test_body=test.fire_multi(type_xpath_multi,xpath_multi_1,xpath_multi_2,table_selector_multi)
test_df = pd.DataFrame(test_body,columns=columns_multi)
test_df.to_csv(index=False,encoding='utf-8-sig',path_or_buf=r'C:\Users\Administrator\Desktop\새 폴더\fire_multi_20170901_20170930.csv')

In [ ]:
#건물유형 테스트
test=nfds_crawling(chrome,url_bldg,'2017-09-01','2017-09-30')
test.date_creation()
test_body=test.fire_bldg(type_xpath_bldg,xpath_bldg_1,xpath_bldg_2,table_selector_bldg)
test_df = pd.DataFrame(test_body,columns=columns_bldg)
test_df.to_csv(index=False,encoding='utf-8-sig',path_or_buf=r'C:\Users\Administrator\Desktop\새 폴더\fire_bldg_20170901_20170930.csv')

In [ ]:
#시도 Test
js.get(url_area)
test=nfds_crawling(js,url_area,'2017-09-01','2017-09-30')
test.date_creation()
test_body=test.area_table(xpath_area,table_selector_area)
test_df = pd.DataFrame(test_body,columns=columns_area)
test_df.to_csv(index=False,encoding='utf-8-sig',path_or_buf=r'C:\Users\Administrator\Desktop\새 폴더\fire_area_20170901_20170930.csv')

In [ ]:
#소방관서 Test
chrome.get(url_station)
test=nfds_crawling(chrome,url_station,'2017-09-01','2017-09-30')
test.date_creation()
test_body=test.area_table(xpath_station_1,table_selector_station)
test_df = pd.DataFrame(test_body,columns=columns_area)
test_df.to_csv(index=False,encoding='utf-8-sig',path_or_buf=r'C:\Users\Administrator\Desktop\새 폴더\fire_station_20170901_20170930.csv')